In [1]:
# app.py
from flask import Flask, render_template, request
from kafka import KafkaConsumer
import json
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

app = Flask(__name__)

consumer = KafkaConsumer('netflix-data', bootstrap_servers=['localhost:9092'],
                         value_deserializer=lambda x: json.loads(x.decode('utf-8')))

# Function to train a simple recommendation model
def train_recommendation_model(data):
    tfidf_vectorizer = TfidfVectorizer(stop_words='english')
    tfidf_matrix = tfidf_vectorizer.fit_transform(data['description'].fillna(''))

    cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

    return cosine_sim

df = pd.read_csv(r"F:/BDAT SEM 1/Data Systems Architecture/Final Project/Netflix/data/netflix_titles.csv")
recommendation_model = train_recommendation_model(df)

@app.route('/')
def index():
    messages = []
    filter_director = request.args.get('directorFilter')
    filter_cast = request.args.get('castFilter')

    for message in consumer:
        messages.append(message.value)
        if len(messages) >= 10:
            break

    recommendations = extract_recommendations(messages, filter_director, filter_cast)

    return render_template('index.html', messages=messages, recommendations=recommendations,
                           filter_director=filter_director, filter_cast=filter_cast)

def extract_recommendations(messages, filter_director=None, filter_cast=None):
    recommendations = []

    for message in messages:
        if (not filter_director or message.get("director") == filter_director) and \
           (not filter_cast or message.get("cast") == filter_cast):
            message['recommendations'] = [df['title'][i] for i in recommendation_model[df.index].argsort()[-6:-1][::-1]]
            recommendations.append(message)

    return recommendations

In [2]:
if __name__ == '__main__':
    try:
        app.run(debug=False)
    except Exception as e:
        print("An error occurred:", str(e))

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [11/Dec/2023 15:21:28] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [11/Dec/2023 15:21:28] "GET /static/styles/style.css HTTP/1.1" 304 -
127.0.0.1 - - [11/Dec/2023 15:21:28] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [11/Dec/2023 15:21:55] "GET /?directorFilter=B.+T+Thomas&castFilter=None HTTP/1.1" 200 -
127.0.0.1 - - [11/Dec/2023 15:21:55] "GET /static/styles/style.css HTTP/1.1" 304 -
127.0.0.1 - - [11/Dec/2023 15:23:13] "GET /?directorFilter=B.+T+Thomas&castFilter=None HTTP/1.1" 200 -
127.0.0.1 - - [11/Dec/2023 15:23:13] "GET /static/styles/style.css HTTP/1.1" 304 -
127.0.0.1 - - [11/Dec/2023 15:23:28] "GET /?directorFilter=Thomas&castFilter=None HTTP/1.1" 200 -
127.0.0.1 - - [11/Dec/2023 15:23:28] "GET /static/styles/style.css HTTP/1.1" 304 -
127.0.0.1 - - [11/Dec/2023 15:23:29] "GET /?directorFilter=Thomas&castFilter=None HTTP/1.1" 200 -
127.0.0.1 - - [11/Dec/2023 15:23:29] "GET /static/styles/style.css HTTP/1.1" 3